# 高维Vector
本章继续对Vector进行扩展，以Vector2D为基础定义多维向量，希望多维向量支持以下功能：  
1.实现基本的序列协议\__len__和\__getitem__  
2.正确展示超高维向量  
3.切片支持，用于生成新的Vector对象  
4.基于\__format__实现新的格式语言扩展
## 序列式构造
序列类型的构造方法一般接受可迭代的对象为参数，因此新的Vector没有采用类似于Vector2D的Vector2D(3, 4)的构造方式。

In [16]:
from array import array
import reprlib
import math
import functools
import numbers
import itertools
import operator

class Vector:
    typecode = 'd'
    # 声明的便捷属性
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        # 这里没有使用双下划线来使用python的名称修改机制，仅用单下划线声明_components为私有（或受保护的），但个人认为这种方式
        # 不如使用双下划线，因为即便子类使用同名属性，python的改名机制会使得两种属性都同时存在于子类实例中。
        self._components = array(self.typecode, components)
    
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        # 如果维度超过6希望利用...表示，因此这里使用reprlib.repr，然后做了进一步的处理来展示。也可使用类似于下面str的方式交给list处理。
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def _str__(self):
        # 这里使用tuple来展示，tuple的str方法会对超过6个的元素做省略展示，如果Vector分量很多，需要复制全部分量创建元组会有一定开销。
        return str(tuple(self))
    
    def __bytes__(self):
        # 记录typecode的ascii编码+将对象转为array.array类型后再进行字节化
        # array.array默认以8B双精度浮点数存储每一个元素，因此共16个字节 + 额外标记'd'共17字节
        return (bytes(self.typecode, 'ascii') + bytes(array(self.typecode, self._components)))
    
    def __eq__(self, other):
        # 这里使用迭代比较的方式确定长度以及各个分量相等，而不是使用简单的构造tuple后交给元组判断相等，因为当分量较多时创建元组使得效率低下。
        return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))
    
    def __hash__(self):
        # 利用归约函数实现对各个分量的hash值求异或。
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self.components))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self.components)
    
    def __getitem__(self, index):
        cls = type(self)
        # 如果传入的是切片，则利用array.array对切片的支持啊重新创建一个Vector对象来模拟内置序列的行为。
        # 如果没有内置的序列来提供对切片的支持，则可能需要使用slice对象的indices(len)方法生成标准的起、始、步长参数。
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            # 抛出的异常应当和内置序列类型的相同。
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))
            
    def __getattr__(self, name):
        # 当属性查找失败后会调用此方法，因此当使用‘xyzt’得到元素时就会调用此方法去得到相应的坐标值。
        # 另一方面要控制实例创建‘xyzt’属性或对其进行赋值。
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        # 对试图添加'a' to 'z'的属性加以限制。如果是添加‘xyzt’这些已定义的快结属性，则提示只读。
        # 对实例属性的限制不能使用__slots__加以限制，__slots__只用于属性存储优化。
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attribute 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name = cls.__name__, attr_name = name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endwith('h'):
            fmt_spec = fmt_spec[:-1]
            # itertools.chain可连接不同的可迭代对象得到总的生成器表达式。
            coords = itertools.chain([abs(self)] + self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        # 生成表达式，当分量较多时可有效减少内存占用。
        components = (format(c, fmt_spec) for c in coods)
        return outer_fmt.format(', '.join(components))
    
    @classmethod
    def frombytes(cls, octets):
        # 将序列化的对象第一个字节转为字符，得到对象存储的单位标识。
        typecode = chr(octets[0])
        # 以单位标识读取剩余字节对应的内存空间，并使用存储的格式（默认使用的是双精度浮点数，因此每次memoryview以8B读取）对其进行读取。
        memv = memoryview(octets[1:]).cast(typecode)
        # 直接新建对象
        return cls(memv)

In [15]:
# 重新整顿slice的起、始、步长，返回元组。
slice(*slice(None, 10, 3).indices(5))

slice(0, 5, 3)

In [6]:
# 省略分量
v1 = Vector(range(10))
str(v1)

'Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])'